In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        import tqdm
import torch
from sklearn.metrics import confusion_matrix
from tabulate import tabulate
# import transformers as ppb # pytorch transformers
from sklearn.metrics import precision_recall_fscore_support

from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer

import matplotlib.pyplot as plt


from pathlib import Path
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nandu-project/artists-data.csv
/kaggle/input/nandu-project/lyrics-data.csv


In [2]:
artist = Path("../input/nandu-project/artists-data.csv")
songs = Path("../input/nandu-project/lyrics-data.csv")

In [3]:
df_artist = pd.read_csv (artist)
df_songlist  =  pd.read_csv (songs)
df_new = df_artist.rename(columns={'Link': 'ALink'})
## Here we are only going to take the ENGLSIH Songs.
new_df_songlist = df_songlist[df_songlist['Idiom'] == 'ENGLISH']
## Merging both the dataframes based on a common link ALink
df_merge = pd.merge(new_df_songlist,df_new,on = 'ALink')
## Check for null values 
df_merge.isnull().sum()
# Removal of null values
df_merge.dropna(inplace = True) 

In [4]:
df_merge.columns

Index(['ALink', 'SName', 'SLink', 'Lyric', 'Idiom', 'Artist', 'Songs',
       'Popularity', 'Genre', 'Genres'],
      dtype='object')

In [5]:
df_merge  = df_merge.drop(labels = ['ALink', 'SName', 'SLink','Idiom', 'Artist', 'Songs',
       'Popularity','Genres'], axis=1)

In [6]:
df_merge = df_merge.reset_index(drop=True)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [8]:
model = AutoModel.from_pretrained('bert-base-uncased')
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NameError: name 'device' is not defined

In [ ]:
class SongDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        lyrics = self.df.loc[idx,'Lyric']
        labels = self.df.loc[idx,'Genre']
        return lyrics, labels

In [ ]:
torch_df = SongDataset(df=df_merge, tokenizer=tokenizer)

In [ ]:
dataloader = DataLoader(torch_df, shuffle=False, batch_size=200, num_workers=0, pin_memory=True)

In [ ]:
device =  'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
arr = np.empty((0,768))
for data in dataloader:
    tokens = tokenizer(data[0], truncation=True, padding=True)
    input_ids = torch.tensor(tokens['input_ids']).to(device)
    attn_mask = torch.tensor(tokens['attention_mask']).to(device)
    with torch.no_grad():
        lhs = model(input_ids, attn_mask).last_hidden_state
        lhs_np = lhs.cpu().numpy()
        bool_mask = ~np.array(tokens['attention_mask']).astype(bool)
        bool_mask = np.repeat(bool_mask, lhs_np.shape[-1], axis=-1)
        bool_mask = bool_mask.reshape(lhs_np.shape)
        masked_mean = np.ma.array(lhs_np, mask=bool_mask).mean(axis=1)
        arr = np.append(arr=arr, values=masked_mean.data, axis=0)

In [ ]:
np.save('contextual_embeddings.npy', arr)

In [ ]:
df_merge.to_csv("./merged_df", index=False)